## 1. What is the underlying concept of Support Vector Machines?

“Support Vector Machine” (SVM) is a supervised machine learning algorithm which can be used for both classification or regression challenges. However, it is mostly used in classification problems. In the SVM algorithm, we plot each data item as a point in n-dimensional space (where n is number of features you have) with the value of each feature being the value of a particular coordinate. Then, we perform classification by finding the hyper-plane that differentiates the two classes very well. Support Vectors are simply the co-ordinates of individual observation. The SVM classifier is a frontier which best segregates the two classes (hyper-plane/ line).

## 2. What is the concept of a support vector?

Support vectors are data points that are closer to the hyperplane and influence the position and orientation of the hyperplane. Using these support vectors, we maximize the margin of the classifier. Deleting the support vectors will change the position of the hyperplane. These are the points that help us build our SVM.

## 3. When using SVMs, why is it necessary to scale the inputs?

Because Support Vector Machine (SVM) optimization occurs by minimizing the decision vector w, the optimal hyperplane is influenced by the scale of the input features and it’s therefore recommended that data be standardized (mean 0, var 1) prior to SVM model training. Advantage of scaling is to avoid attributes in greater numeric ranges dominating those in smaller numeric ranges. Another advantage is to avoid numerical difficulties during the kernel calculation.

## 4. When an SVM classifier classifies a case, can it output a confidence score? What about a percentage chance?

An SVM classifier can give the distance between the test instance and the decision boundary as output, so we can use that as a confidence score, but we cannot use this score to directly converted it into class probabilities. But if you set probability=True when building a model of SVM in Scikit-Learn, then after training it will calibrate the probabilities using Logistic Regression on the SVM’s scores. By using this techniques, we can add the predict_proba() and predict_log_proba() methods to the SVM model.

## 5. Should you train a model on a training set with millions of instances and hundreds of features using the primal or dual form of the SVM problem?

This question applies only to linear SVMs since kernelized can only use the dual form. The computational complexity of the primal form of the SVM problem is proportional to the number of training instances m, while the computational complexity of the dual form is proportional to a number between m2 and m3. So if there are millions of instances, you should definitely use the primal form, because the dual form will be much too slow. The dual problem is faster to solve than the primal when the number of training instances is smaller than the number of features

## 6. Let's say you've used an RBF kernel to train an SVM classifier, but it appears to underfit the training collection. Is it better to raise or lower (gamma)? What about the letter C?

There might be too much regularization. To decrease it, you need to increase gamma or C (or both). Increasing gamma makes the bell-shape curve narrower, and as a result each instance's range of influence is smaller: the decision boundary ends up being more irregular, wiggling around individual instances. Conversely, a small gamma value makes the bell-shaped curve wider, so instances have a larger range of influence, and the decision boundary ends up smoother. A smaller C value leads to a wider street but more margin violations.

## 7. To solve the soft margin linear SVM classifier problem with an off-the-shelf QP solver, how should the QP parameters (H, f, A, and b) be set?

We set the QP parameters for the hard margin problem as H, f, A, and b. The QP parameters for the soft margin problem will have m additional parameters and m additional constrains.

Thus, H is equal to H plus m columns of 0s on the right and m rows of 0s at the bottom. f is euqal to f with m additional elements all equal to the value of hyperparameter C. A is equal to A with extra m*m identity matrix appended to the right. b is equal to b with m additional elements all equal to 0.


## 8. On a linearly separable dataset, train a LinearSVC. Then, using the same dataset, train an SVC and an SGDClassifier. See if you can get them to make a model that is similar to yours.


Of course it depends on the dataset and of course a lot of other factors add weight but today in this small post I’ll demonstrate how to use LinearSVC , SVC classifier and SGD classifier via Python code and also compare results for the same dataset. Since classifiers are very sensitive to outliers we need to scale them but before we need to pick the right dataset, for simpler results I’ll showcase the benchmark dataset for all classifiers — the Iris dataset.

## 9. On the MNIST dataset, train an SVM classifier. You'll need to use one-versus-the-rest to assign all 10 digits because SVM classifiers are binary classifiers. To accelerate up the process, you might want to tune the hyperparameters using small validation sets. What level of precision can you achieve?

In [2]:
from sklearn.datasets import fetch_openml
import numpy as np
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

Many training algorithms are sensitive to the order of the training instances, so it's generally good practice to shuffle them first. However, the dataset is already shuffled, so we do not need to do it.

Let's start simple, with a linear SVM classifier. It will automatically use the One-vs-All (also called One-vs-the-Rest, OvR) strategy, so there's nothing special we need to do. Easy!

Warning: this may take a few minutes depending on your hardware.

In [4]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train, y_train)

C:\Users\amitp\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

Let's make predictions on the training set and measure the accuracy (we don't want to measure it on the test set yet, since we have not selected and trained the final model yet):

In [5]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.8348666666666666

Okay, 83.5% accuracy on MNIST is pretty bad. This linear model is certainly too simple for MNIST, but perhaps we just needed to scale the data first:

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

Warning: this may take a few minutes depending on your hardware.

In [7]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train_scaled, y_train)

C:\Users\amitp\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

In [8]:
y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9214

That's much better (we cut the error rate by about 53%), but still not great at all for MNIST. If we want to use an SVM, we will have to use a kernel. Let's try an SVC with an RBF kernel (the default).

Note: to be future-proof we set gamma="scale" since it will be the default value in Scikit-Learn 0.22.

In [9]:
svm_clf = SVC(gamma="scale")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])

SVC()

In [10]:
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9455333333333333

That's promising, we get better performance even though we trained the model on 6 times less data. Let's tune the hyperparameters by doing a randomized search with cross validation. We will do this on a small dataset just to speed up the process:

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ..C=10.681547960095235, gamma=0.0028158224555557415; total time=   0.1s
[CV] END ..C=10.681547960095235, gamma=0.0028158224555557415; total time=   0.1s
[CV] END ..C=10.681547960095235, gamma=0.0028158224555557415; total time=   0.1s
[CV] END ....C=2.925568364659578, gamma=0.004161140909269049; total time=   0.1s
[CV] END ....C=2.925568364659578, gamma=0.004161140909269049; total time=   0.2s
[CV] END ....C=2.925568364659578, gamma=0.004161140909269049; total time=   0.1s
[CV] END ....C=8.59408271797526, gamma=0.0032594242889414397; total time=   0.1s
[CV] END ....C=8.59408271797526, gamma=0.0032594242889414397; total time=   0.1s
[CV] END ....C=8.59408271797526, gamma=0.0032594242889414397; total time=   0.1s
[CV] END ....C=1.9669720862883824, gamma=0.09882261086465988; total time=   0.1s
[CV] END ....C=1.9669720862883824, gamma=0.09882261086465988; total time=   0.2s
[CV] END ....C=1.9669720862883824, gamma=0.09882

RandomizedSearchCV(cv=3, estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EB91C048E0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EB91BF6DC0>},
                   verbose=2)

In [12]:
rnd_search_cv.best_estimator_

SVC(C=10.681547960095235, gamma=0.0028158224555557415)

In [13]:
rnd_search_cv.best_score_

0.8229756702810596

This looks pretty low but remember we only trained the model on 1,000 instances. Let's retrain the best estimator on the whole training set:

Warning: the following cell may take hours to run, depending on your hardware.

In [14]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

SVC(C=10.681547960095235, gamma=0.0028158224555557415)

In [15]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9999666666666667

Ah, this looks good! Let's select this model. Now we can test it on the test set:

In [16]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.9664

Not too bad, but apparently the model is overfitting slightly. It's tempting to tweak the hyperparameters a bit more (e.g. decreasing C and/or gamma), but we would run the risk of overfitting the test set. Other people have found that the hyperparameters C=5 and gamma=0.005 yield even better performance (over 98% accuracy). By running the randomized search for longer and on a larger part of the training set, you may be able to find this as well.

## 10. On the California housing dataset, train an SVM regressor.

In [17]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
X = housing["data"]
y = housing["target"]

Split it into a training set and a test set:

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Don't forget to scale the data:

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Let's train a simple LinearSVR first:

In [20]:
from sklearn.svm import LinearSVR

lin_svr = LinearSVR(random_state=42)
lin_svr.fit(X_train_scaled, y_train)

C:\Users\amitp\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVR(random_state=42)

Let's see how it performs on the training set:

In [21]:
from sklearn.metrics import mean_squared_error

y_pred = lin_svr.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
mse

0.9641780189948642

In [22]:

np.sqrt(mse)

0.9819256687727764

In this training set, the targets are tens of thousands of dollars. The RMSE gives a rough idea of the kind of error you should expect (with a higher weight for large errors): so with this model we can expect errors somewhere around $10,000. Not great. Let's see if we can do better with an RBF Kernel. We will use randomized search with cross validation to find the appropriate hyperparameter values for C and gamma:

In [23]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(SVR(), param_distributions, n_iter=10, verbose=2, cv=3, random_state=42)
rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=   9.6s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=   9.2s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=   9.0s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   8.6s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   8.8s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   8.9s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   8.6s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   8.7s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   8.9s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=   8.2s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=   8.2s
[CV] END ....C=1.5808361216819946, gamma=0.05399

RandomizedSearchCV(cv=3, estimator=SVR(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EB91C33370>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EB91C04460>},
                   random_state=42, verbose=2)

In [24]:

rnd_search_cv.best_estimator_

SVR(C=4.745401188473625, gamma=0.07969454818643928)

Now let's measure the RMSE on the training set:

In [25]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)

0.5727524770785369

Looks much better than the linear model. Let's select this model and evaluate it on the test set:

In [26]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)

0.5929168385528745